In [81]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import datetime
import pandas as pd
import datetime
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 

In [2]:
data = {"dates" : [],
        "values" : []
        }

In [4]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )          
             
        counter+=1

        if counter == 5:
            counter = 0
            iter += 1
            avg_y = (landmarks[15].y+landmarks[16].y+landmarks[17].y+landmarks[18].y+
                        landmarks[19].y+landmarks[20].y+landmarks[21].y+landmarks[22].y)/8

            data["values"].append(avg_y)
            data["dates"].append(datetime.today().strftime("%d-%m-%y %H:%M:%S"))
            if iter == 180:
                break
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [18]:
df_raw = pd.DataFrame(data = data)

In [19]:
df_raw.to_csv("y_axis_bench.csv" , index = False)
df_raw = df_raw.drop(index=range(28))

In [20]:
df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [21]:
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')
df_raw.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [266]:

import math

df['Speed'] = df['values'].diff() / df['values'].count()
times = pd.Series(datetime.strptime(df['dates'][0],"%d-%m-%y %H:%M:%S"))
df['Rest_Time'] = times.diff()
df['Intensity'] = (df['Speed'] + df['values']) / 2
df['Fatigue'] = df['Intensity'].diff()

s = df['Speed'].values

results = []
counter = 0
for i in range(len(s)-1):
    s[i] *= 100000
    if math.isnan(s[i]):
        s[i] = 0
    if abs(s[i]) < 60:
        counter+=1
    else:
        if abs(s[i-1]) < 60 :
            results.append(counter)
            counter=0

print(results)

[1, 1, 2, 5, 2, 1, 1, 1, 1, 5, 2, 4, 2, 4, 3, 1, 3, 4, 4, 3, 3, 4]


C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speed'] = df['values'].diff() / df['values'].count()
C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rest_Time'] = times.diff()
C:\Users\mosac\AppData\Local\Temp\ipykernel_15096\1774298178.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [264]:
results

[1, 1, 2, 5, 2, 1, 1, 1, 1, 5, 2, 4, 2, 4, 3, 1, 3, 4, 4, 3, 3, 4]

In [23]:
df

,dates,values
28,27-02-23 15:20:01,0.366482
29,27-02-23 15:20:02,0.409691
34,27-02-23 15:20:03,0.352312
36,27-02-23 15:20:04,0.138342
37,27-02-23 15:20:04,0.108396
...,...,...
201,27-02-23 15:20:47,0.398203
202,27-02-23 15:20:47,0.477402
203,27-02-23 15:20:48,0.500039
204,27-02-23 15:20:48,0.547422


The Use Of SARIMA - Seasonal Autoregressive Integrated Moving Average

In [63]:
order = (1, 1, 1)
seasonal_order = (1, 0, 1, 12)

# fit SARIMA model to time series
model = sm.tsa.statespace.SARIMAX(df['values'], order=order, seasonal_order=seasonal_order)
results = model.fit()

# make predictions for next 30 seconds (assuming last observation was at 15:20:30)
start_time = pd.to_datetime(df['dates'][df.index[-1]]) + pd.Timedelta(seconds=1)
end_time = start_time + pd.Timedelta(seconds=30)
forecast = results.predict(start=start_time, end=end_time)

print(forecast)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [84]:
train = df[:round(len(df)*0.7)]
test = df[round(len(df)*0.7):]
test.head()

,dates,values,forecast
161,27-02-23 15:20:37,0.044600,0.306299
163,27-02-23 15:20:37,0.098223,0.307552
164,27-02-23 15:20:38,0.109677,0.310124
167,27-02-23 15:20:38,0.386639,0.307823
168,27-02-23 15:20:39,0.425768,0.306243


In [117]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)

# fit SARIMA model to time series
model = sm.tsa.statespace.SARIMAX(train["values"], order=order, seasonal_order=seasonal_order)
results = model.fit()

# make predictions for next 30 seconds (assuming last observation was at 15:20:30)
start_time = df.index[-1] + 1
end_time = start_time + 30
forecast = results.predict(start=test.index[0], end=test.index[-1])
df["forecast"] = forecast
print(forecast)

IndexError: tuple index out of range

In [109]:
pd.DataFrame(forecast).to_csv('1.csv')
